In [2]:
import nltk
from nltk.corpus import webtext
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import math as m
ids = nltk.corpus.webtext.fileids()
N = len(ids)
ps = PorterStemmer()
dict={}
from nltk.corpus import wordnet

In [3]:
def text_preprocessing(text):
    text_tokenize = word_tokenize(text)
    text_tokenize = [word.lower() for word in text_tokenize]
    word_filtered = [word for word in text_tokenize if not word in stopwords.words('english')]
    word_lem = [lemmatizer.lemmatize(word) for word in word_filtered]
    return text_tokenize , word_filtered , word_lem

In [4]:
def Documents_preprocessing():
    docs={}
    all_words = []
    for id in ids:
        with open(nltk.corpus.webtext.abspath(id),'r', encoding='utf-8', errors='ignore') as file:
            text = file.read()
            text_tokenize , word_filtered , word_lem = text_preprocessing(text)
            word_count = {}
            for word in word_filtered:
                word_count[word] = word_count.get(word,0) + 1
            data = [text_tokenize,word_filtered,word_lem,word_count]
            docs[id] = data
            all_words += text_tokenize
    return docs , all_words

In [5]:
from collections import defaultdict
def Indexation(all_words,docs):
    ind= defaultdict(lambda : {})
    for id , data in docs.items():
        text_tokenize , word_filtered , word_lem ,word_count = data
        for word , count in word_count.items():
            tf = count / len(word_filtered)
            document_frequency = sum(word in value[1]  for key , value in docs.items())
            freq = count
            idf = m.log(N/(document_frequency + 1))
            poids = (1+m.log10(freq))*(m.log10(N/document_frequency))
            tf_idf = tf*idf
            ind[word][id] = {'tf':tf,'frequency':freq,'tf_idf':tf_idf,'idf':idf,'poids':poids}
    return ind

In [6]:
def Recherche(mot,ind,all_words):
    documents = []
    poids = []
    tfidf = []
    freq = []
    mot_lem = lemmatizer.lemmatize(mot)
    if mot_lem in all_words:
        for doc_id , word_ind in ind[mot_lem].items():
            documents.append(doc_id)
            poids.append(word_ind['poids'])
            tfidf.append(word_ind['tf_idf'])
            freq.append(word_ind['frequency'])
            
    return documents , poids , tfidf , freq

In [7]:
def definition(word):
  syns = wordnet.synsets(word)
  if (len(syns) == 0):
    dfn =""
  else:
    dfn = syns[0].definition()
  return dfn

In [8]:
def synant(word):
  synonyms = []
  antonyms = []
  syns = wordnet.synsets(word)
  for syn in syns:
    for l in syn.lemmas():
      synonyms.append(l.name())
      if l.antonyms():
        antonyms.append(l.antonyms()[0].name())
  return synonyms, antonyms

In [9]:
def fonction_grammaticale(word):
  tokens = nltk.word_tokenize(word)
  tags = nltk.pos_tag(tokens)
  fg = [tag[1] for tag in tags]
  return fg

In [10]:
docs , all_words = Documents_preprocessing()